# Python security best practices


## Use Python 3

Although Python 3 has been out for more than decade, many people and companies are still running Python 2.7 in production. As of the time of this writing, Python 2.7 is still officially supported. The Python Software Foundation has announced  that support for ,<strong>Python 2 ends January 1, 2020</strong>. If you have not upgraded by then, you leave yourself open to security vulnerabilities, both within the language and within other open source projects that are unlikely to maintain compatibility with Python 2.7.

For instance, Django 1.11 is the last version of Django that is compatible with Python 2.7. Long term support for Django 1.11 is promised through at least April 2020, but it would not be wise to rely on security support beyond that point.

The transition to Python 3 has not been easy for the community. The breaking changes introduced in Python 3 mean that a software developer needs to be sure that their legacy codebases are ready for upgrade, and also that all of their open source dependencies are compatible with Python 3.

If you are still running Python 2.7, now is the time to prioritize your technical debt and upgrade.

## Scan your code with Bandit

A simple way to find security vulnerabilities within your Python code is to run a scan with [Bandit](https://pypi.org/project/bandit/).|

Bandit is an open source project that is available through the Python Packaging Index (PyPI). Bandit scans each .py file and builds a corresponding abstract syntax tree (AST). Bandit then runs a number of plugins against the AST to find common software security problems. For example, one plugin can detect whether you are using Flask (a micro-framework for Python) with the debug setting equal to True.

Bandit works either as a local tool to be used as you develop, or as part of your CI/CD (continuous integration/ continuous delivery)  pipeline. You can create a YAML configuration file to control how Bandit behaves in these different scenarios. In this file you can also indicate a list of tests to skip. This functionality should be used with caution.

There is no guarantee that Bandit will catch all security problems—there are a finite number of plugins that it runs, and you could potentially have an issue in your code that doesn’t register against any of the available plugins. However, it is easy to use and an excellent screen for common issues.

### How to install Bandit

In [ ]:
!pip install bandit

### How to run a Scan with Bandit

In [2]:
!bandit -r ./scripts/

Run started:2019-08-15 17:15:11.971210

Test results:
>> Issue: [B105:hardcoded_password_string] Possible hardcoded password: 'mypassword123'
   Severity: Low   Confidence: Medium
   Location: ./scripts/code_to_test_bandit.py:3
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b105_hardcoded_password_string.html
2	#Bandit examples
3	password='mypassword123'
4	
5	name = raw_input("What is your name?")

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Medium
   Location: ./scripts/code_to_test_bandit.py:6
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
5	name = raw_input("What is your name?")
6	cur.execute("SELECT * FROM Students WHERE Name = '%s';" % name)
7	

--------------------------------------------------
>> Issue: [B501:request_with_no_cert_validation] Requests call with v

[main]	INFO	profile include tests: None
[main]	INFO	profile exclude tests: None
[main]	INFO	cli include tests: None
[main]	INFO	cli exclude tests: None
[main]	INFO	running on Python 3.7.1



## Use Pipenv for environment and dependency management


The ideally the developer’s local environment should be identical to the production environment in order to eliminate surprises. It can therefore be tempting to run pip freeze on your local machine, dump the resulting list of packages and versions into a requirements.txt, and then use that file to set up your production environment. This is easy to do, but not the most security conscious option.

When you pin your dependencies, you freeze your project to a moment in time. This is great for predictability, but leaves your project exposed as new security vulnerabilities are found and remediated for those open source dependencies.

Pipenv is a tool that manages the competing interests of having a predictable environment and having an up-to-date environment. It uses a two-file system that separates abstract dependency declarations from the last tested combination. Pipenv manages your installations and your virtual environment, displays your dependency tree, and can check your dependencies for known vulnerabilities.

## Watch your import statements
Python imports are very flexible, but that flexibility has a security cost.

When importing in Python, you can use an absolute import or a relative import. An absolute import uses the entire path (starting at the root directory) of the module that you want to import. If the module you want to import is not found at that location, an error occurs. Absolute imports are a good way to know exactly what you are importing.

A relative import starts at the path of the current module. There are two types of relative imports, explicit and implicit. Explicit relative imports specify the precise location of the module you want to import with respect to the current module. For example, you might have an import statement that looks like this `from .. import my_module`. The dots indicate how many directories to traverse upwards.

An implicit relative import does not specify a location relative to the current module. If the module is found in the system path it is imported, which could be dangerous. It could be possible to create a malicious module with the same name as a popular module and then smuggle it into a popular open source library. If the malicious module is found in the system path before the real module it is imported instead.

<strong>Import statements in Python execute the code in the imported module—this means that an implicit relative import could result in the execution of malicious code. For this reason, implicit relative imports are not supported in Python 3.</strong>

If you are using Python 2, eliminate the use of implicit relative imports. This is important for the current security of your project and because it is a necessary step towards upgrading to Python 3. If you are using Python 3, it is still important to keep in mind that import statements execute the code within the target module. Because of this, it makes sense to be careful with your import statements, regardless of the Python version that you are using.

## Be careful with string formatting

Despite Python’s ideal of having one and only one way to do things, it actually has four different ways to format strings (three methods for versions prior to Python 3.6).

String formatting has gotten progressively more flexible and powerful (f-strings are particularly interesting) but as flexibility increases, so does the potential for exploits. For this reason, Python users should carefully consider how they format strings with data entered by users.

Python has a built-in module named string. This module includes the Template class, which is used to create template strings.

Consider the following example.

In [ ]:
from string import Template
greeting_template = Template('Hello World, my name is $name.')
greeting = greeting_template.substitute(name='Pedro')
greeting

This string format is a bit cumbersome because it requires an import statement and it is less flexible with types. It also doesn’t evaluate Python statements the way f-strings do. These constraints make template strings an excellent choice when dealing with user input.

Another quick note on string formatting: Be extra careful with raw SQL. Make your queries with object-relational mapping (ORM) if at all possible.

In [ ]:
name = input("What is your name?")
cur.execute("SELECT * FROM Students WHERE Name = %s;", (name))     

## Review your dependency licenses

When considering using an open source project, it is important to understand how these projects are licensed. Open source projects are free and available to use, but there may still be terms and conditions applied. These terms usually involve how the software is used, whether you need to make any changes you make to the software publicly available, and other similar requirements. You should become familiar with the licenses necessary for the projects you use, so you are sure that you are not compromising yourself legally.

Snyk recently reviewed the project licenses available through PyPI. More than 10% of the available packages do not specify a license. These packages may have the appearance of open source software (they are freely available), but they are not explicitly open source.

If the project adopts a more restrictive license than you anticipated, you have essentially cornered yourself. You should comply to the terms of the license or cease using the project. Additionally, if you need to make changes to the project that does not have a license, you might run afoul with copyright law.

To ensure that your project is sustainable and to protect yourself, know what licenses your dependencies use and comply with the terms.

----------
Hayley Denbraver, Kenneth Reitz (February, 2019) Python Security Best Practices Cheat Sheet. Retireved from https://snyk.io/blog/python-security-best-practices-cheat-sheet/

PyCQA team (July 2019) bandit 1.6.2. Retrieved from https://pypi.org/project/bandit/